In [1]:
import syft as sy
import numpy as np
import jax

/home/teo/anaconda3/envs/PySyft/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-05 09:21:26.380031: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64:/usr/local/cuda-11.5/lib64:
2022-12-05 09:21:26.412308: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-05 09:21:27.146199: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory; L

In [2]:
def func1(x,y):
    return 2*x + y

def func2(x,y):
    return func1(x,y) * x

grad_fn1 = jax.grad(func1)
grad_fn2 = jax.grad(func2)

print(grad_fn1(2., 1.))
print(grad_fn2(2., 1.))

2.0
9.0


In [2]:
data = np.array([2., 1.])
phi_tensor = sy.Tensor(data).annotate_with_dp_metadata(lower_bound=1, upper_bound=2, data_subjects="ishan")
gamma_tensor = (phi_tensor[0] * 2 + phi_tensor[1]) * phi_tensor[0] 
# gamma_tensor = (phi_tensor * phi_tensor + phi_tensor) * phi_tensor

Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.


In [4]:
from jax import numpy as jnp
from scipy.optimize import shgo
from typing import Deque
from collections import deque

def create_new_lookup_tables(
    dictionary: dict,
):
    index2key: Deque = deque()
    key2index: dict = {}
    index2values: Deque = (
        deque()
    )  # Note this maps to GammaTensor, not to GammaTensor.child as name may imply
    index2size: Deque = deque()
    for index, key in enumerate(dictionary.keys()):
        key = str(key)
        index2key.append(key)
        key2index[key] = index
        index2values.append(dictionary[key])
        index2size.append(len(dictionary[key]))

    return index2key, key2index, index2values, index2size

def lipschitz_bound(tensor, data_subject) -> float:
    # TODO: Check if there are any functions for which lipschitz bounds shouldn't be computed
    # if dis(self.func) == dis(no_op):
    #     raise Exception

    print("Starting JAX JIT")
    # relative
    from syft.core.tensor.autodp.gamma_functions import GAMMA_FUNC_MAPPER
    from syft.core.tensor.autodp.gamma_tensor_ops import GAMMA_TENSOR_OP
    from syft.core.adp.vectorized_publish import get_leaves_from_gamma_tensor_tree
    
    # TODO: this must be composed out of the whole computation tree
    fn = jax.jit(GAMMA_FUNC_MAPPER[GAMMA_TENSOR_OP(tensor.func_str)])
    # print(fn)
    # print("Traced self.func with jax's jit, now calculating gradient")
    grad_fn = jax.grad(fn)
    # phi_tensors = get_leaves_from_gamma_tensor_tree(tensor)
    
    # # this function will receive the data as one dimensional array
    # def max_grad_fn(input_values: np.ndarray) -> float:
    #     vectors = {}
        
    #     # we need to add the variable vectors belonging to the data_subject 
    #     # we are trying to minimize with regard to, and also the constant vectors
    #     # that belong to the other data_subjects

    #     grad_pred = grad_fn(vectors)

    #     m = 0
    #     for value in grad_pred.values():
    #         m = max(m, jnp.max(value))

    #     # return negative because we want to maximize instead of minimize
    #     return -m

    # print("starting SHGO")
    # res = shgo(max_grad_fn, bounds)
    # print("Ran SHGO")
    # # return negative because we flipped earlier
    # return -float(res.fun)
    return grad_fn

In [5]:

from syft.core.tensor.autodp.gamma_functions import GAMMA_FUNC_MAPPER
from syft.core.tensor.autodp.gamma_tensor_ops import GAMMA_TENSOR_OP
gamma_tensor.child.sources.values()

dict_values([GammaTensor(child=5.0, data_subjects=DataSubjectArray: {'ishan'}, min_vals=<lazyrepeatarray data: 5.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 5.0 -> shape: ()>, is_linear=True, func_str='add', id='102850160', sources={'1216026388': GammaTensor(child=4.0, data_subjects=array(DataSubjectArray: {'ishan'}, dtype=object), min_vals=<lazyrepeatarray data: 4.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 4.0 -> shape: ()>, is_linear=True, func_str='noop', id='1216026388', sources={}), '276557268': GammaTensor(child=1.0, data_subjects=array(DataSubjectArray: {'ishan'}, dtype=object), min_vals=<lazyrepeatarray data: 1.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 1.0 -> shape: ()>, is_linear=True, func_str='noop', id='276557268', sources={})}), GammaTensor(child=2.0, data_subjects=array(DataSubjectArray: {'ishan'}, dtype=object), min_vals=<lazyrepeatarray data: 2.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 2.0 -> shape: ()>, is_linear=True, func_str='noop', id='9290

In [6]:
from syft.core.adp.vectorized_publish import get_leaves_from_gamma_tensor_tree
phi_tensors = get_leaves_from_gamma_tensor_tree(gamma_tensor.child)
print([phi.child for phi in phi_tensors])
vectors = {}
for phi in phi_tensors:
    vectors[phi.id] = phi.child
print(vectors)
fun = lipschitz_bound(gamma_tensor.child, None)
fun._fun
gamma_tensor.child
# fun(vectors)

[4.0, 1.0, 2.0]
{'1216026388': 4.0, '276557268': 1.0, '929086921': 2.0}
Starting JAX JIT


GammaTensor(child=10.0, data_subjects=DataSubjectArray: {'ishan'}, min_vals=<lazyrepeatarray data: 10.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 10.0 -> shape: ()>, is_linear=True, func_str='multiply', id='2081560700', sources={'102850160': GammaTensor(child=5.0, data_subjects=DataSubjectArray: {'ishan'}, min_vals=<lazyrepeatarray data: 5.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 5.0 -> shape: ()>, is_linear=True, func_str='add', id='102850160', sources={'1216026388': GammaTensor(child=4.0, data_subjects=array(DataSubjectArray: {'ishan'}, dtype=object), min_vals=<lazyrepeatarray data: 4.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 4.0 -> shape: ()>, is_linear=True, func_str='noop', id='1216026388', sources={}), '276557268': GammaTensor(child=1.0, data_subjects=array(DataSubjectArray: {'ishan'}, dtype=object), min_vals=<lazyrepeatarray data: 1.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 1.0 -> shape: ()>, is_linear=True, func_str='noop', id='276557268', sources={})

In [7]:
phi_tensors

[GammaTensor(child=4.0, data_subjects=array(DataSubjectArray: {'ishan'}, dtype=object), min_vals=<lazyrepeatarray data: 4.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 4.0 -> shape: ()>, is_linear=True, func_str='noop', id='1216026388', sources={}),
 GammaTensor(child=1.0, data_subjects=array(DataSubjectArray: {'ishan'}, dtype=object), min_vals=<lazyrepeatarray data: 1.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 1.0 -> shape: ()>, is_linear=True, func_str='noop', id='276557268', sources={}),
 GammaTensor(child=2.0, data_subjects=array(DataSubjectArray: {'ishan'}, dtype=object), min_vals=<lazyrepeatarray data: 2.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 2.0 -> shape: ()>, is_linear=True, func_str='noop', id='929086921', sources={})]

SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
from .gamma_tensor_ops import GAMMA_TENSOR_OP

ops = {
    GAMMA_TENSOR_OP.NOOP: lambda x: x,
    GAMMA_TENSOR_OP.ADD: jnp.add,
    GAMMA_TENSOR_OP.SUBTRACT: jnp.subtract,
    GAMMA_TENSOR_OP.MULTIPLY: jnp.multiply,
    GAMMA_TENSOR_OP.TRUE_DIVIDE: jnp.true_divide,
    GAMMA_TENSOR_OP.MATMUL: jnp.matmul,
    GAMMA_TENSOR_OP.RMATMUL: make_r_infix_op(jnp.matmul),
    GAMMA_TENSOR_OP.GREATER: jnp.greater,
    GAMMA_TENSOR_OP.GREATER_EQUAL: jnp.greater_equal,
    GAMMA_TENSOR_OP.EQUAL: jnp.equal,
    GAMMA_TENSOR_OP.NOT_EQUAL: jnp.not_equal,
    GAMMA_TENSOR_OP.LESS: jnp.less,
    GAMMA_TENSOR_OP.LESS_EQUAL: jnp.less_equal,
    GAMMA_TENSOR_OP.LOG: jnp.log,
    GAMMA_TENSOR_OP.TRANSPOSE: jnp.transpose,
    GAMMA_TENSOR_OP.SUM: jnp.sum,
    GAMMA_TENSOR_OP.ONES_LIKE: jnp.ones_like,
    GAMMA_TENSOR_OP.ZEROS_LIKE: jnp.zeros_like,
    GAMMA_TENSOR_OP.RAVEL: jnp.ravel,
    GAMMA_TENSOR_OP.RESIZE: jnp.resize,
    GAMMA_TENSOR_OP.RESHAPE: jnp.reshape,
    GAMMA_TENSOR_OP.COMPRESS: jnp.compress,
    GAMMA_TENSOR_OP.SQUEEZE: jnp.squeeze,
    GAMMA_TENSOR_OP.ANY: jnp.any,
    GAMMA_TENSOR_OP.ALL: jnp.all,
    GAMMA_TENSOR_OP.LOGICAL_AND: jnp.logical_and,
    GAMMA_TENSOR_OP.LOGICAL_OR: jnp.logical_or,
    GAMMA_TENSOR_OP.POSITIVE: jnp.positive,
    GAMMA_TENSOR_OP.NEGATIVE: jnp.negative,
    GAMMA_TENSOR_OP.MEAN: jnp.mean,
    GAMMA_TENSOR_OP.STD: jnp.std,
    GAMMA_TENSOR_OP.VAR: jnp.var,
    GAMMA_TENSOR_OP.DOT: jnp.dot,
    GAMMA_TENSOR_OP.CUMSUM: jnp.cumsum,
    GAMMA_TENSOR_OP.CUMPROD: jnp.cumprod,
    GAMMA_TENSOR_OP.SQRT: jnp.sqrt,
    GAMMA_TENSOR_OP.ABS: jnp.abs,
    GAMMA_TENSOR_OP.CHOOSE: jnp.choose,
    GAMMA_TENSOR_OP.CLIP: jnp.clip,
    GAMMA_TENSOR_OP.COPY: jnp.copy,
    GAMMA_TENSOR_OP.TAKE: jnp.take,
    GAMMA_TENSOR_OP.PUT: jnp.put,
    GAMMA_TENSOR_OP.ARGMAX: jnp.argmax,
    GAMMA_TENSOR_OP.ARGMIN: jnp.argmin,
    GAMMA_TENSOR_OP.PTP: jnp.ptp,
    GAMMA_TENSOR_OP.MOD: jnp.mod,
    GAMMA_TENSOR_OP.SWAPAXES: jnp.swapaxes,
    GAMMA_TENSOR_OP.NONZERO: jnp.nonzero,
    GAMMA_TENSOR_OP.PROD: jnp.prod,
    GAMMA_TENSOR_OP.FLOOR_DIVIDE: jnp.floor_divide,
    GAMMA_TENSOR_OP.POWER: jnp.power,
    GAMMA_TENSOR_OP.TRACE: jnp.trace,
    GAMMA_TENSOR_OP.MIN: jnp.min,
    GAMMA_TENSOR_OP.MAX: jnp.max,
    GAMMA_TENSOR_OP.LSHIFT: jnp.left_shift,
    GAMMA_TENSOR_OP.RSHIFT: jnp.right_shift,
    GAMMA_TENSOR_OP.XOR: jnp.bitwise_xor,
    GAMMA_TENSOR_OP.ROUND: jnp.round,
    GAMMA_TENSOR_OP.SORT: jnp.sort,
    GAMMA_TENSOR_OP.ARGSORT: jnp.argsort,
    GAMMA_TENSOR_OP.REPEAT: jnp.repeat,
    GAMMA_TENSOR_OP.DIVMOD: jnp.divmod,
    GAMMA_TENSOR_OP.FLATTEN_C: get_flatten_type(order="C"),
    GAMMA_TENSOR_OP.FLATTEN_F: get_flatten_type(order="F"),
    GAMMA_TENSOR_OP.FLATTEN_A: get_flatten_type(order="A"),
    GAMMA_TENSOR_OP.FLATTEN_K: get_flatten_type(order="K"),
}